In [1]:
import numpy as np
import math
import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
%matplotlib inline

In [2]:
def GellMann(idx):
    """
    outputs the Gell-Mann matrix associated with given idx
        idx=0 returns identity
    uses convention of https://en.wikipedia.org/wiki/Gell-Mann_matrices
    """
    if(idx == 0):
        return np.eye(3)
    elif(idx == 1):
        return np.array([
            [0, 1, 0],
            [1, 0, 0],
            [0, 0, 0]
        ])
    elif(idx == 2):
        return np.array([
            [0, -1j, 0],
            [1j, 0, 0],
            [0, 0, 0]
        ])
    elif(idx == 3):
        return np.array([
            [1, 0, 0],
            [0, -1, 0],
            [0, 0, 0]
        ])
    elif(idx == 4):
        return np.array([
            [0, 0, 1],
            [0, 0, 0],
            [1, 0, 0]
        ])
    elif(idx == 5):
        return np.array([
            [0, 0, -1j],
            [0, 0, 0],
            [1j, 0, 0]
        ])
    elif(idx == 6):
        return np.array([
            [0, 0, 0],
            [0, 0, 1],
            [0, 1, 0]
        ])
    elif(idx == 7):
        return np.array([
            [0, 0, 0],
            [0, 0, -1j],
            [0, 1j, 0]
        ])
    elif(idx == 8):
        return (1/np.sqrt(3))*np.array([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, -2]
        ])
    else: 
        print("ERROR! GellMann(idx): \n index out of bounds. ")
        return 0
    
    
def GellMannQ(idx):
    """Return qutip qobj of Gell-Mann matrix"""
    return qt.Qobj(GellMann(idx))

# The general state of a q-trit may be written 

$$ \rho = \frac{\mathbb{1} + \sqrt{3} \vec{r} \cdot \vec{\gamma}}{2} $$
where $\vec{r}$ is an 8-component vector on $\mathbb{R}$ and $\gamma$ is an 8-component vector of the b $SU(3)$


A general 3-level Hamiltonian is similarly written
$$
    H = w \mathbb{1} + \vec{v} \cdot \vec{\gamma}
$$


In [3]:
def density3(coList):
    """
    returns a 3-level density state with coherence vector 
        given by coList ("coherence list")
    """
    rho = 0.5*np.eye(3)
    N = np.sqrt(3)/2
    for idx, co in enumerate(coList):
        rho = rho + N*co*GellMann(idx+1) # +1 needed since GellMann(0) = id
    return rho

def Ham3(spectList):
    """
    returns a 3-level Hamiltonian with 
        given spectrum in SU(3)
    """
    H = 0
    for idx, s in enumerate(spectList):
        H = H + s*GellMann(idx)
    return H

def Spectrum3(Op):
    """
    returns the spectrum of operator Op in SU(3)
    """
    specOut = [0.5*np.trace(Op@GellMann(i)) for i in range(9)]
    return specOut

In [4]:
def SpinObservable(idx, spect):
    """
    Converts between an SU(3) spectrum and a spin observables with the following equivalences
    
    assuming hbar = 1
    
    idx 0: <Sz>
    idx 1: <Sy> 
    idx 2: <Sx> 
    idx 3: <Sy^2>
    idx 4: <Sx^2>
    idx 5: <{Sz, Sx}>
    idx 6: <{Sy, Sz}>
    idx 7: <{Sx, Sy}>
    
    Reference: page 11322 of J. Phys. A: Math. Gen. 39 (2006) 11313–11324
    doi:10.1088/0305-4470/39/36/012
    """
    if(idx == 0): # Sz
        return spect[3] + np.sqrt(3)*spect[8]
    elif(idx == 1): # Sy
        return np.sqrt(2)*(spect[2] + spect[7])
    elif(idx == 2): # Sx
        return np.sqrt(2)*(spect[1] + spect[6])
    elif(idx == 3): # Sy^2
        return 0.5*(4/3 - spect[3] - 2*spect[4] + spect[8]/np.sqrt(3))
    elif(idx == 4): # Sx^2
        return 0.5*(4/3 - spect[3] + 2*spect[4] + spect[8]/np.sqrt(3))
    elif(idx == 5): # {Sz, Sx}
        return np.sqrt(2)*(spect[1] - spect[6])
    elif(idx == 6): # {Sy, Sz}
        return np.sqrt(2)*(spect[2] - spect[7])
    elif(idx == 7): # {Sx, Sy}
        return 2*spect[5]

In [5]:
# some examples of usage

rho1 = density3([0,0,0,0,0,0,0,1])

cvec = [2, 1, 1, 1, 0,1, 0, 0, 1]

H1 = Ham3(cvec)
print(H1)

print(np.trace(H1*GellMann(0)))
print(Spectrum3(H1))

[[3.57735027+0.j 1.        -1.j 0.        -1.j]
 [1.        +1.j 1.57735027+0.j 0.        +0.j]
 [0.        +1.j 0.        +0.j 0.84529946+0.j]]
(6+0j)
[(3+0j), (1+0j), (1+0j), (1+0j), 0j, (1+0j), 0j, 0j, (1+0j)]


# solve and plot

takes an input 3-level pure state 
$$
    \begin{pmatrix}
    N \\
    b_1 e^{i \theta_1} \\
    b_2 e^{i \theta_2} 
    \end{pmatrix}
$$

In [45]:
def SolveAndPlot(T, DeltaT, b1, b2, theta1, theta2, H0, H1, H2, H3, H4, H5, H6, H7, H8, L0, L1, L2, L3, L4, L5, L6, L7, L8):
    """
    Function wrapper of the code used for animation
    """
    # time vector
    tvec = np.linspace(0, T, DeltaT)
    
    # calculate psi0
    if((b1**2 + b2**2) > 1): # warn if spectral power exceeds maximum 
        print("ERROR! SolveAndPlot::calculate psi0: \n spectral power exceeds ceiling!")
        N = 0
    else:
        N = np.sqrt(1 - b1**2 - b2**2)
    
    psi0 = (
        N*qt.basis(3,0) + 
        b1*np.exp(1j*2*np.pi*theta1/360)*qt.basis(3,1) + 
        b2*np.exp(1j*2*np.pi*theta2/360)*qt.basis(3,2)
    )
    
    # define Linblad operators 
    Lops = [ 
        L0*GellMannQ(0),
        L1*GellMannQ(1),
        L2*GellMannQ(2),
        L3*GellMannQ(3),
        L4*GellMannQ(4),
        L5*GellMannQ(5),
        L6*GellMannQ(6),
        L0*GellMannQ(7),
        L0*GellMannQ(8),
        ]
    
    
    
    # calculate H
    H = qt.Qobj(
        H0*GellMann(0) + 
        H1*GellMann(1) + 
        H2*GellMann(2) + 
        H3*GellMann(3) + 
        H4*GellMann(4) + 
        H5*GellMann(5) + 
        H6*GellMann(6) + 
        H7*GellMann(7) + 
        H8*GellMann(8)
    )
    
    # solve
    ExpVec = qt.mesolve(H, psi0, tvec, Lops, [GellMannQ(0), GellMannQ(1), GellMannQ(2), GellMannQ(3), GellMannQ(4), GellMannQ(5), GellMannQ(6), GellMannQ(7), GellMannQ(8)])
    result = qt.mesolve(H, psi0, tvec, Lops, [])
    
    # process 
    RT = range(int(DeltaT))
    RhoVec = [result.states[t] for t in RT]
    # SU(3)
    Gset = []
    for l in range(9):
        Gset.append([(RhoVec[t]*GellMann(l)).trace() for t in RT])
    # Spin Observables 
    Sset = []
    for s in range(8):
        Sset.append(
        [
            SpinObservable(s, [Gset[0][t], Gset[1][t], Gset[2][t], Gset[3][t], Gset[4][t], Gset[5][t], Gset[6][t], Gset[7][t], Gset[8][t]])
            for t in RT
        ]
        )
    # Eigenvalues
    EigSet = [RhoVec[t].eigenenergies() for t in RT]
    
    # plot
    fig, ax = plt.subplots(3)
    for l in range(9):
        ax[0].plot(Gset[l])
    ax[0].set_title('SU(3) Generators')
    ax[0].set_xlabel('Time');
    ax[0].set_ylabel('Expectation values');
    ax[0].legend(["GL(0)", "GL(1)", "GL(2)", "GL(3)", "GL(4)", "GL(5)", "GL(6)", "GL(7)", "GL(8)"], bbox_to_anchor=(1.25, 2));
    
    
    for s in range(8):
        ax[1].plot(Sset[s])
    ax[1].set_title('Spin Observables')
    ax[1].set_xlabel('Time');
    ax[1].set_ylabel('Expectation values');
    ax[1].legend(["Sz", "Sy", "Sx", "Sy^2", "Sx^2", "{Sz, Sx}", "{Sy, Sz}", "{Sx, Sy}"], bbox_to_anchor=(1.25, 1))
    
    ax[2].plot(EigSet)
    ax[2].set_title('Eigen-Energies')
    ax[2].set_xlabel('Time');
    ax[2].set_ylabel('Energy');
    ax[2].legend(["E1", "E2", "E3"], bbox_to_anchor=(1.25, 0));
    
    
    
    ax[1].label_outer()
    ax[0].label_outer()
    # ax.set_box_aspect((1, 1, 1)) 
    # ax = a3.Axes3D(plt.figure(figsize=(15,15))) 
    plt.show(fig)

In [46]:
_T = FloatSlider(min=1, max=100, step=1)
_DeltaT = FloatSlider(min=1, max=400, step=1)

_b1 = FloatSlider(min=0, max=1, step=0.05)
_b2 = FloatSlider(min=0, max=1, step=0.05)

_theta1 = FloatSlider(min=0, max=360, step=1)
_theta2 = FloatSlider(min=0, max=360, step=1)

_H0= FloatSlider(min=0, max=10, step=0.5)
_H1 = FloatSlider(min=0, max=10, step=0.5)
_H2 = FloatSlider(min=0, max=10, step=0.5)
_H3 = FloatSlider(min=0, max=10, step=0.5)
_H4 = FloatSlider(min=0, max=10, step=0.5)
_H5 = FloatSlider(min=0, max=10, step=0.5)
_H6 = FloatSlider(min=0, max=10, step=0.5)
_H7 = FloatSlider(min=0, max=10, step=0.5)
_H8 = FloatSlider(min=0, max=10, step=0.5)

_L0 = FloatSlider(min=0, max=1, step=0.05)
_L1 = FloatSlider(min=0, max=1, step=0.05)
_L2 = FloatSlider(min=0, max=1, step=0.05)
_L3 = FloatSlider(min=0, max=1, step=0.05)
_L4 = FloatSlider(min=0, max=1, step=0.05)
_L5 = FloatSlider(min=0, max=1, step=0.05)
_L6 = FloatSlider(min=0, max=1, step=0.05)
_L7 = FloatSlider(min=0, max=1, step=0.05)
_L8 = FloatSlider(min=0, max=1, step=0.05)

interact_manual(SolveAndPlot, T=_T, DeltaT=_DeltaT, b1=_b1, b2=_b2, theta1=_theta1, theta2=_theta2, H0=_H0, H1=_H1, H2=_H2, H3=_H3, H4=_H4, H5=_H5, H6=_H6, H7=_H7, H8=_H8,
                L0=_L0, L1=_L1, L2=_L2, L3=_L3, L4=_L4, L5=_L5, L6=_L6, L7=_L7, L8=_L8)

interactive(children=(FloatSlider(value=1.0, description='T', min=1.0, step=1.0), FloatSlider(value=1.0, descr…

<function __main__.SolveAndPlot(T, DeltaT, b1, b2, theta1, theta2, H0, H1, H2, H3, H4, H5, H6, H7, H8, L0, L1, L2, L3, L4, L5, L6, L7, L8)>

# TODO: make display for eigenvalues of density matrix, normalized eigenvectors, and Spin Observables